In [1]:
import os

In [2]:
%pwd

'f:\\zizo\\MLOPS\\datascienceproject\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\zizo\\MLOPS\\datascienceproject'

In [5]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )
        return data_transformation_config



In [8]:
import os
from src.datascience import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def Preprocessing(self):
        data = pd.read_csv(self.config.data_path)
        data = data.drop(["STATION","TAVG_ATTRIBUTES","NAME"],axis= 1)

        data['DATE'] = pd.to_datetime(data['DATE'])
        data['YEAR'] = data['DATE'].dt.year
        data['MONTH'] = data['DATE'].dt.month
        data['DAY'] = data['DATE'].dt.day
        data['DAY_OF_WEEK'] = data['DATE'].dt.dayofweek
        data['DAY_OF_YEAR'] = data['DATE'].dt.dayofyear
        data = data.drop('DATE', axis = 1)
        return data
    
    def train_test_splitting(self,data: pd.DataFrame):

        train,test = train_test_split(data)
        train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index=False)
        test.to_csv(os.path.join(self.config.root_dir,"test.csv"),index = False)


        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)
        print(train.shape)
        print(test.shape)
        """scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(data)

        sequence_length = 10
        num_features = len(data.columns)

        sequences = []
        labels = []
        for i in range(len(scaled_data) - sequence_length):
            seq = scaled_data[i:i+sequence_length]
            label = scaled_data[i+sequence_length][3]
            sequences.append(np.hstack((seq, np.full((sequence_length, 1), label))))

        sequences = np.array(sequences)

        train_size = int(0.8 * len(sequences))
        train, test = sequences[:train_size], sequences[train_size:]

        print("Train Data shape:", train.shape)
        print("Test Data shape:", test.shape)
        train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index=False)
        test.to_csv(os.path.join(self.config.root_dir,"test.csv"),index = False)
        
        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)"""


        




In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    preprocessing_data = data_transformation.Preprocessing()
    data_transformation.train_test_splitting(preprocessing_data)
except Exception as e:
    raise e

[2025-04-15 13:09:26,605: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-15 13:09:26,607: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-15 13:09:26,609: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-15 13:09:26,610: INFO: common: created directory at: artifacts]
[2025-04-15 13:09:26,610: INFO: common: created directory at: artifacts/data_transformation]
[2025-04-15 13:09:26,986: INFO: 3618622370: Splited data into training and test sets]
[2025-04-15 13:09:26,986: INFO: 3618622370: (73834, 9)]
[2025-04-15 13:09:26,987: INFO: 3618622370: (24612, 9)]
(73834, 9)
(24612, 9)
